# импорты

In [97]:
from pydantic import BaseModel
import pandas as pd
import json
from tqdm import tqdm

In [69]:
class AttributeDescription(BaseModel):
    attribute: str  # Любой атрибут, например "вкус", "запах", "текстура"
    characteristic: str  # Его описание, например "сладкий", "свежий", "мягкий"

class ProductReviewDataPoint(BaseModel):
    id_tov: str
    name_tov: str
    name_par_group: str
    attribute_descriptions: list[AttributeDescription]

class ReviewContent(BaseModel):
    data_points: list[ProductReviewDataPoint]


In [88]:
# OPENAI_MODEL="gpt-4o-2024-08-06"
# OPENAI_MODEL="gpt-4o-mini" 
OPENAI_MODEL="gpt-4o"

# сочетания промптов и моделей

__вариант 1 gpt-4o-mini__

"Ты помощник, который анализирует отзывы на товары. "
"Из каждого отзыва извлекай атрибуты продукта (например, вкус, запах, текстура, цвет) "
"и их описания (например, сладкий, свежий, мягкий, сочный). "
"Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
"Возвращай результат в формате: список атрибутов и их описаний."

__вариант 2 gpt-4o-mini__

"Ты помощник, который анализирует отзывы на товары. "
"Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
"Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
"Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
"Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
"Возвращай результат в формате: список атрибутов и их описаний."

__вариант 3 gpt-4o__

"Ты помощник, который анализирует отзывы на товары. "
"Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
"Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
"Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
"Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
"Возвращай результат в формате: список атрибутов и их описаний."

__вариант 4 gpt-4o-mini__
"Ты помощник, который анализирует отзывы на товары. "
"Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
"У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
"Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
"Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:"
"Очень вкусные, сладкие! Никакие абхазские не нужны))"
"Огромные, почти безвкусные"
"Хорошие мандарины, в меру сладкие, без косточек"
"Нам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!"
"Вкусные свежие"
"Мандарины без косточек, легко чистятся, в основном сладкие. По кислости попадаются разные, лично мне нравятся те, что покислее. Даже в одном мандарине иногда часть долек кислее других - странно. Иногда попадаются подсушенные мандарины, как будто их переморозили, но в основном мандарины хорошие."
"Неплохие, но немного кисловаты."
"сладкие, косточек не попалось, кожура тонкая и легко чистится"
"Спасибо, тонкокорые,сочные ,сладкие,очень понравились"
"Мандарины очень сладкие. Вкусные"
"Любимые мандарины"
"Безвкусные. Иран"
"из этого списка отзывов выделяем следующие атрибуты и их характеристики:"
"вкус: сладкий, безвкусный"
"размер: огромный"
"кожура: тонкая и легко чистится"
"структура: без косточек, легко чистятся"
"текстура: сочные, подсушенные"
"Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
"Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
"Возвращай результат в формате: список атрибутов и их описаний."

__вариант 5 gpt-4o__
"Ты помощник, который анализирует отзывы на товары. "
"Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
"У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
"Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
"Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:"
"Очень вкусные, сладкие! Никакие абхазские не нужны))"
"Огромные, почти безвкусные"
"Хорошие мандарины, в меру сладкие, без косточек"
"Нам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!"
"Вкусные свежие"
"Мандарины без косточек, легко чистятся, в основном сладкие. По кислости попадаются разные, лично мне нравятся те, что покислее. Даже в одном мандарине иногда часть долек кислее других - странно. Иногда попадаются подсушенные мандарины, как будто их переморозили, но в основном мандарины хорошие."
"Неплохие, но немного кисловаты."
"сладкие, косточек не попалось, кожура тонкая и легко чистится"
"Спасибо, тонкокорые,сочные ,сладкие,очень понравились"
"Мандарины очень сладкие. Вкусные"
"Любимые мандарины"
"Безвкусные. Иран"
"из этого списка отзывов выделяем следующие атрибуты и их характеристики:"
"вкус: сладкий, безвкусный"
"размер: огромный"
"кожура: тонкая и легко чистится"
"структура: без косточек, легко чистятся"
"текстура: сочные, подсушенные"
"Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
"Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
"Возвращай результат в формате: список атрибутов и их описаний."

# генерим синтетику

In [89]:
def extract_review_attributes(id_tov, 
name_tov, 
name_par_group,
repr_text
):
    system_prompt = (
        "Ты помощник, который анализирует отзывы на товары. "
        "Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
        "У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
        "Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
        "Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:"
        "Очень вкусные, сладкие! Никакие абхазские не нужны))"
        "Огромные, почти безвкусные"
        "Хорошие мандарины, в меру сладкие, без косточек"
        "Нам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!"
        "Вкусные свежие"
        "Мандарины без косточек, легко чистятся, в основном сладкие. По кислости попадаются разные, лично мне нравятся те, что покислее. Даже в одном мандарине иногда часть долек кислее других - странно. Иногда попадаются подсушенные мандарины, как будто их переморозили, но в основном мандарины хорошие."
        "Неплохие, но немного кисловаты."
        "сладкие, косточек не попалось, кожура тонкая и легко чистится"
        "Спасибо, тонкокорые,сочные ,сладкие,очень понравились"
        "Мандарины очень сладкие. Вкусные"
        "Любимые мандарины"
        "Безвкусные. Иран"
        "из этого списка отзывов выделяем следующие атрибуты и их характеристики:"
        "вкус: сладкий, безвкусный"
        "размер: огромный"
        "кожура: тонкая и легко чистится"
        "структура: без косточек, легко чистятся"
        "текстура: сочные, подсушенные"
        "Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
        "Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
        "Возвращай результат в формате: список атрибутов и их описаний."
    )

    from openai import OpenAI
    client = OpenAI()

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": repr_text
        },
    ]

    response = client.beta.chat.completions.parse(
        model=OPENAI_MODEL,
        messages=messages,
        response_format=ReviewContent
    )

    data_points = [
        {
            "id_tov": id_tov,
            "name_tov": name_tov,
            "name_par_group": name_par_group,
            "attribute_descriptions": [
                {
                    "attribute": x.attribute,
                    "characteristic": x.characteristic
                }
                for x in response.choices[0].message.parsed.data_points[0].attribute_descriptions
            ]
        }
    ]

    return data_points

In [ ]:
#это файл с товарами и отзывами на них, из отзывов будем генерить атрибуты и их характеристики
reviews_df = pd.read_csv('attr_10388.csv')
print(reviews_df.shape)
reviews_df.head()

In [142]:
# имя выходного файла
output_file = "product_attributes_10k.json" 

# создаем файл или очищаем его, если он уже существует
with open(output_file, "w") as json_file:
    json_file.write("[")

output_data = []
batch_size = 100 
first_batch = True  # Флаг для первой записи

# обработка каждого отзыва
for i, row in tqdm(reviews_df.iterrows(), total=len(reviews_df)):
    try:
        id_tov = row["id_tov"]
        name_tov = row["name_tov"]
        name_par_group = row["name_par_group"]
        repr_text = row["repr_text"]

        structured_data = extract_review_attributes(
            id_tov,
            name_tov,
            name_par_group,
            repr_text
        )

        # результаты в общий список
        output_data.append(structured_data[0])  

    except Exception as e:
        print(f"Ошибка обработки отзыва для товара {row['id_tov']}: {e}")

    # обновляем файл каждые batch_size записей
    if (i + 1) % batch_size == 0 or i == len(reviews_df) - 1:
        with open(output_file, "a") as json_file:
            if not first_batch:
                json_file.write(",")  # добавляем запятую между батчами
            else:
                first_batch = False  # после первого батча запятая не нужна

            json.dump(output_data, json_file, indent=4, ensure_ascii=False)
            output_data = []  # очищаем временный список


with open(output_file, "a") as json_file:
    json_file.write("]")

 32%|███▏      | 3348/10388 [1:53:53<56:04:43, 28.68s/it]

Ошибка обработки отзыва для товара 58080: 1 validation error for ReviewContent
  Invalid JSON: EOF while parsing a string at line 1 column 2 [type=json_invalid, input_value='{"', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid


100%|██████████| 10388/10388 [5:27:08<00:00,  1.89s/it]  


In [143]:
with open(output_file, "r") as json_file:
    json_data = json.load(json_file)

print("JSON файл успешно прочитан.")

JSON файл успешно прочитан.


In [144]:
reviews_df.shape

(10388, 4)

In [149]:
#убираем пропущенную строку в json - костыль, что-то не то с одним из товаров
reviews_df = reviews_df[reviews_df.id_tov != 58080]

In [151]:
reviews_df.isna().sum()

id_tov                    0
name_tov                  0
name_par_group            0
repr_text                 0
attribute_descriptions    0
dtype: int64

In [145]:
with open(output_file, "r") as json_file:
    final_data = json.load(json_file)

if isinstance(final_data[0], list):
    final_data = [item for batch in final_data for item in batch]

attribute_descriptions = [json.dumps(item, ensure_ascii=False) for item in final_data]

In [150]:
reviews_df["attribute_descriptions"] = attribute_descriptions
reviews_df.to_excel('prod_attr_10k.xlsx', index=False)
# эксель нужен для валидации пользователями
print(f"Структурированные данные сохранены в {output_file} и Excel файл prod_attr_10k.xlsx.")

Структурированные данные сохранены в product_attributes_10k.json и Excel файл prod_attr_10k.xlsx.
